# RFM Analysis for ABC Superstore

This notebook performs RFM (Recency, Frequency, Monetary) analysis on historical sales data to segment customers.

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load the dataset
file_path = 'Sample - Superstore.xlsx'
df = pd.read_excel(file_path)
df.head()

In [ ]:
# Convert Order Date to datetime
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Customer ID'] = df['Customer ID'].astype(str)

In [ ]:
# Snapshot date (last date in the data + 1 day)
snapshot_date = df['Order Date'].max() + pd.Timedelta(days=1)

# RFM calculation
rfm = df.groupby('Customer ID').agg({
    'Order Date': lambda x: (snapshot_date - x.max()).days,
    'Order ID': 'nunique',
    'Sales': 'sum'
})
rfm.columns = ['Recency', 'Frequency', 'Monetary']
rfm = rfm.reset_index()
rfm.head()

In [ ]:
# RFM scoring (1 to 5)
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5,4,3,2,1]).astype(int)
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5]).astype(int)
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 5, labels=[1,2,3,4,5]).astype(int)

# Combine scores
rfm['RFM_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)
rfm.head()

In [ ]:
# Define segments (example rule-based mapping)
def segment_customer(df):
    if df['R_Score'] >= 4 and df['F_Score'] >= 4:
        return 'Champion'
    elif df['R_Score'] >= 3 and df['F_Score'] >= 3:
        return 'Loyal'
    elif df['R_Score'] >= 4:
        return 'Recent'
    elif df['F_Score'] >= 4:
        return 'Frequent'
    else:
        return 'Others'

rfm['Segment'] = rfm.apply(segment_customer, axis=1)
rfm['Segment'].value_counts()